In [ ]:
#cell 1 — imports & setup
"""
this first cell downloads openstreetmap road network data for hoboken and converts 
multigraph to simple weighted digraph for pathfinding 

NOTICE: do not change anything in this cell, the project works specifically for hoboken, if 
you try another locationit will not run properly
"""

import os
import pickle
import folium
import matplotlib.pyplot as plt


from maploader import get_graph_for_place, graph_to_simple_weighted, nearest_node, geocode_address, get_nearest_node
from planner import PathPlanner


#choose a place
place = "Hoboken, New Jersey, USA"
G_osm = get_graph_for_place(place)
G = graph_to_simple_weighted(G_osm)

#initialize path planner with preprocessed graph
planner = PathPlanner(G)
print(planner)

In [ ]:
#cell 2 - user input and pathfinding
"""
converts street addresses to coordinates, maps them to graph nodes, and runs pathfinding algorithm
"""

#sample inputs:
#221 Washington St, Hoboken, NJ 07030
#1320 Adams St Unit C, Hoboken, NJ 07030
#1000 Park Ave, Hoboken, NJ 07030


# User input
address_start = input("Enter start address: ")
address_end = input("Enter destination address: ")

# 1. geocode the addresses
start_lat, start_lon = geocode_address(address_start)
end_lat, end_lon = geocode_address(address_end)

print("Start coordinates:", start_lat, start_lon)
print("End coordinates:", end_lat, end_lon)

# 2. convert coordinates to nearest graph nodes
start_node = get_nearest_node(G_osm,start_lat, start_lon)
end_node = get_nearest_node(G_osm,end_lat, end_lon)

print("Mapped to graph nodes:", start_node, end_node)


#run pathfinding and display results
path, total_m, t = planner.plan(start_node, end_node)
print(f"Path length: {total_m:.1f} m, nodes: {len(path)}, time: {t:.3f}s")

In [ ]:
#cell 3 - route visualization

"""
creates interactive folium map displaying the calculated path with start/end markers
"""

m = folium.Map(location=[(start_lat+end_lat)/2, (start_lon+end_lon)/2], zoom_start=14)
coords = [(G.nodes[n]["y"], G.nodes[n]["x"]) for n in path]
folium.PolyLine(coords, weight=6, opacity=0.8).add_to(m)
folium.Marker(coords[0], tooltip="Start").add_to(m)
folium.Marker(coords[-1], tooltip="Goal").add_to(m)


m # in notebook this will display